In [1]:
import numpy as np

#x is position of planet
#M is mass of planet
#G is gravitational constant

#function returns gravitational acceleration
def gravAccf(x, G, M):
    return -G*M/(np.linalg.norm(x)**3)*x

def nBodyF(t, solVec, mass):
    G = 4*(np.pi)**2
    N = np.size(mass)
    solVec = np.reshape(solVec, (N,6)).T
    F = np.zeros(np.shape(solVec))
    for b in range(0,N):
        for p in range(0,N):
            if b != p:
                F[0:3,[b]] = F[0:3,[b]]-gravAccf(solVec[3:6,[b]]-solVec[3:6,[p]],G,mass[p])
    F = -F
    for b in range(0,N):   
        F[3:6,[b]]=solVec[0:3,[b]]
    func = np.reshape(F.T,(np.size(solVec),1))
    return func

In [2]:
def Betamethod(f, tRange, u0, df, beta, h):
    N = len(u0)
    solArray = u0
    tArray = np.arange(tRange[0], tRange[-1]+h, h)
    #explicit methond
    if beta == 0:
        for k in range(0, len(tArray)-1):
            u0 = u0+h*f(tArray[k],u0)
            solArray = np.append(solArray, u0,1)
    #implicit method        
    else:
        for k in range(0, len(tArray)-1): 
            def F(x):
                return x-u0-h*f(tArray[k]+h*beta, (1-beta)*u0+beta*x)
            def Df(x):
                return np.eye(N)-h*beta*df(tArray[k]+h*beta,(1-beta)*u0+beta*x)   
            [success, xHist, xfin, errEst] = newton(F, Df, u0, tol=10**(-12), maxIt=50)
            u0=xfin
            solArray = np.append(solArray, xfin, 1)
            #terminate if Newton's method has failed
            if success == 0:
                print('newton method failed')
                break
                
    return[tArray, solArray]

In [3]:
def newton(f, df, x0, tol, maxIt):
    
    success = 0
    
    def G(x):
        return x-np.linalg.solve(df(x),f(x))
    
    xHist = x0
    errEst = []
    
    for k in range(0,maxIt):
        xHist = np.append(xHist, G(xHist[:,[k]]),1)
        errest = np.linalg.norm(xHist[:,[k+1]]-xHist[:,[k]])
        errEst = np.append(errEst, errest)
        if errest <tol:
            success = 1
            break
            
    xfin = xHist[:,[-1]]
    
    return [success, xHist, xfin, errEst]

In [4]:
def gravAccJac(x, G, M):
    return G*M/(np.linalg.norm(x)**3)*(-np.eye(3)+3/(np.linalg.norm(x)**2)*(x@(x.T)))

def rangeVel(i):
    vel = np.arange(6*(i-1),6*(i-1)+3,1)
    return vel

def rangePos(i):
    pos= np.arange(6*(i-1)+3,6*i,1)
    return pos

def nBodyJac(t, solVec, mass):
    G = 4*np.pi**2
    N = np.size(mass)
    jacF = np.zeros((np.size(solVec),np.size(solVec)))
    for b in range(1,N+1):
        for p in range (1,N+1):
            if b != p:
                jacg= gravAccJac(solVec[rangePos(p)]-solVec[rangePos(b)],G,mass[p-1])
                jacF[rangeVel(b)[:,None],rangePos(p)] = jacF[rangeVel(b)[:,None],rangePos(p)]+jacg
                jacF[rangeVel(b)[:,None],rangePos(b)] = jacF[rangeVel(b)[:,None],rangePos(b)]-jacg
        jacF[rangePos(b)[:,None],rangeVel(b)]= np.eye(3)
    return jacF

In [7]:
from scipy import io

mat = io.loadmat('solarSimData.mat')
mass= mat['bodyMass']
velAndPos = mat['velAndPos']
h = 2/(365*24)
beta = 1/2
tRange = [0,1]

def f(t,y):
    return nBodyF(t,y,mass)

def df(t,y):
    return nBodyJac(t,y,mass)

u0 = np.reshape(velAndPos.T,(66,1))

[tArray, solArray] = Betamethod(f, tRange, u0, df, beta, h)

In [8]:
size_t = np.size(tArray)
dme = np.zeros((3,size_t))
dms = np.zeros((3,size_t))
alpha = np.zeros(size_t)

for t in range(0,np.size(tArray)):
    dme= solArray[27:30,[t]]-solArray[21:24,[t]]
    dms= solArray[27:30,[t]]-solArray[3:6,[t]]
    alpha[t] = (dme.T@dms)/((np.linalg.norm(dme,2)*np.linalg.norm(dms,2))) 

In [9]:
def findlocalMaxima(xvals, yvals):
    
    N = np.size(yvals)
    xmax = []
    ymax = []
    
    for t in range(1,N-1):
        if  yvals[t]>yvals[t-1]:
             if yvals[t] >yvals[t+1]:
                X = [xvals[t-1],xvals[t],xvals[t+1]]
                Y = [yvals[t-1], yvals[t], yvals[t+1]]
                c = np.polyfit(X,Y,2)
                x_max = -c[1]/(2*c[0])
                y_max = c[0]*x_max**2+c[1]*x_max+c[2]
                xmax = np.append(xmax, x_max)
                ymax = np.append(ymax, y_max)
                
    return [xmax, ymax]

In [10]:
import datetime
start = datetime.datetime(2019,5,29,9,0,0)

[xmax, ymax] = findlocalMaxima(tArray,alpha)

for i in range(0, np.size(ymax)):
    if abs(ymax[i]-1) <3*10**(-4):
        lunar = start +datetime.timedelta(seconds=365*24*3600*xmax[i])
        print("lunar eclips at",lunar)
        
[xmax, ymax] = findlocalMaxima(tArray,-alpha)

for i in range(0,np.size(ymax)):
    if abs(-ymax[i]+1) <3*10**(-4):
        solar = start +datetime.timedelta(seconds=365*24*3600*xmax[i])
        print("solar eclips at",solar)

lunar eclips at 2019-07-16 20:18:26.453092
lunar eclips at 2020-01-10 13:47:53.763118
solar eclips at 2019-07-02 18:36:53.961995
solar eclips at 2019-12-26 00:11:03.151452
